

# **Proyecto C: Optimización en la Planificación de Rutas Nacionales y Estrategia de Recarga para Logistico**




**Facultad de ingeniería**

Departamento de Ingeniería de Sistemas y Computación

Universidad de los Andes

**ISIS-3302:** Modelado, Optimización y Simulación

*   

*   Luisa Gabriela Hernández Pérez

*   Adriana Sofia Rozo Cepeda.


In [166]:
!pip install haversine
!pip install networkx matplotlib
!sudo apt-get install -y highs
!pip install highspy

!pip install gurobipy

!pip install pyomo
!apt-get install -y coinor-cbc

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package highs
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
coinor-cbc is already the newest version (2.10.7+ds1-1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


## Implementación del Modelo

In [167]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
from haversine import haversine

solver = SolverFactory('cbc')
print(solver.available())

model = ConcreteModel()


True


Conjuntos

In [168]:
import pandas as pd

# Cargar el archivo CSV
vehicles = pd.read_csv("vehicles.csv")
depots = pd.read_csv("depots.csv")
clients = pd.read_csv("clients.csv")


In [169]:
depositos = {}
for i in range(len(depots)):
  depositos[f"Deposito {int(i+1)}"] = {
      "lat": float(depots["Latitude"].values[i]),
      "lon": float(depots["Longitude"].values[i])
  }

depositos

{'Deposito 1': {'lat': 4.743359, 'lon': -74.153536}}

In [170]:
destinos = {}
for i in range(len(clients)):
  destinos[f"Destino {int(i+1)}"] = {
      "d": float(clients["Demand"].values[i]),
      "lat": float(clients["Latitude"].values[i]),
      "lon": float(clients["Longitude"].values[i])
  }

destinos

{'Destino 1': {'d': 13.0, 'lat': 4.59795431125545, 'lon': -74.09893796560621}, 'Destino 2': {'d': 15.0, 'lat': 4.687820646838871, 'lon': -74.07557103763986}, 'Destino 3': {'d': 12.0, 'lat': 4.70949446000624, 'lon': -74.10708524062704}, 'Destino 4': {'d': 15.0, 'lat': 4.605029068682624, 'lon': -74.09727965657427}, 'Destino 5': {'d': 20.0, 'lat': 4.648463876533332, 'lon': -74.16464148202755}, 'Destino 6': {'d': 17.0, 'lat': 4.662137416953968, 'lon': -74.12083799988112}, 'Destino 7': {'d': 17.0, 'lat': 4.697499030379109, 'lon': -74.02213076607309}, 'Destino 8': {'d': 20.0, 'lat': 4.649416884236942, 'lon': -74.17207549744595}, 'Destino 9': {'d': 20.0, 'lat': 4.606310650273935, 'lon': -74.15615257246444}, 'Destino 10': {'d': 15.0, 'lat': 4.557379705282216, 'lon': -74.09041145358674}, 'Destino 11': {'d': 17.0, 'lat': 4.591594072172954, 'lon': -74.17802255204528}, 'Destino 12': {'d': 12.0, 'lat': 4.7564172406324055, 'lon': -74.1015410917749}, 'Destino 13': {'d': 21.0, 'lat': 4.646217006050524

In [171]:
vehiculos = {}
for i in range(len(vehicles)):
  vehiculos[f"Vehiculo {int(i+1)}"] = {
      "cap": float(vehicles["Capacity"].values[i]),
      "range": float(vehicles["Range"].values[i])
  }

vehiculos

{'Vehiculo 1': {'cap': 130.0, 'range': 170.0}, 'Vehiculo 2': {'cap': 140.0, 'range': 200.0}, 'Vehiculo 3': {'cap': 120.0, 'range': 180.0}, 'Vehiculo 4': {'cap': 100.0, 'range': 90.0}, 'Vehiculo 5': {'cap': 70.0, 'range': 100.0}, 'Vehiculo 6': {'cap': 55.0, 'range': 170.0}, 'Vehiculo 7': {'cap': 110.0, 'range': 150.0}, 'Vehiculo 8': {'cap': 114.0, 'range': 140.0}}

In [172]:
def func_harvesine(ubi1,ubi2):
  return haversine(ubi1,ubi2)


In [173]:
def calculadora_distancias(depositos, destinos):
    nodos = list(depositos.keys()) + list(destinos.keys())
    distancias = {}
    for i in nodos:
        for j in nodos:
            if i != j and not (i in depositos and j in depositos):

                if i in depositos:
                    lat_i, lon_i = depositos[i]["lat"], depositos[i]["lon"]
                else:
                    lat_i, lon_i = destinos[i]["lat"], destinos[i]["lon"]

                if j in depositos:
                    lat_j, lon_j = depositos[j]["lat"], depositos[j]["lon"]
                else:
                    lat_j, lon_j = destinos[j]["lat"], destinos[j]["lon"]

                distancias[(i, j)] = func_harvesine((lat_i,lon_i),(lat_j,lon_j))

    return distancias


In [174]:

distancias = calculadora_distancias(depositos,destinos)
print(distancias)

{('Deposito 1', 'Destino 1'): 17.263444271536358, ('Deposito 1', 'Destino 2'): 10.620126894924754, ('Deposito 1', 'Destino 3'): 6.377821224493921, ('Deposito 1', 'Destino 4'): 16.597119606502737, ('Deposito 1', 'Destino 5'): 10.62340219677577, ('Deposito 1', 'Destino 6'): 9.731264593168408, ('Deposito 1', 'Destino 7'): 15.429106593606146, ('Deposito 1', 'Destino 8'): 10.646038973434123, ('Deposito 1', 'Destino 9'): 15.241861000422377, ('Deposito 1', 'Destino 10'): 21.83131006314039, ('Deposito 1', 'Destino 11'): 17.092320203996398, ('Deposito 1', 'Destino 12'): 5.9418667697568335, ('Deposito 1', 'Destino 13'): 12.492367239367232, ('Deposito 1', 'Destino 14'): 4.004799058951106, ('Deposito 1', 'Destino 15'): 16.81224493507039, ('Deposito 1', 'Destino 16'): 21.207358840856593, ('Deposito 1', 'Destino 17'): 14.533472573067701, ('Deposito 1', 'Destino 18'): 10.188306488493257, ('Deposito 1', 'Destino 19'): 12.205155509363443, ('Deposito 1', 'Destino 20'): 3.5281280083252256, ('Deposito 1',

In [175]:
model.D = Set(initialize=destinos.keys())   # Destinos
model.P = Set(initialize=depositos.keys())   # Puertos
model.V = Set(initialize=vehiculos.keys())  # Vehículos

model.A = Set(within=(model.P | model.D) * (model.P | model.D),
              initialize=[(i, j) for i in (set(depositos.keys()) | set(destinos.keys()))
              for j in (set(depositos.keys()) | set(destinos.keys())) if i != j and not (i in depositos and j in depositos)]
            ) #Arcos


Parametros

In [176]:
distancias_filtradas = {
    (i, j): distancias[(i, j)]
    for (i, j) in model.A
    if (i, j) in distancias
}
model.d = Param(model.A, initialize=distancias_filtradas)

#model.d = Param(model.A,initialize=distancias)                  # Distancia entre nodos
model.cap = Param(model.V, initialize={v: vehiculos[v]["cap"] for v in vehiculos})                # Capacidad del vehículo
model.a = Param(model.V, initialize={v: vehiculos[v]["range"] for v in vehiculos})                  # Autonomía del vehículo
model.m = Param(initialize=350)                         # Costo de mantenimiento por km
model.ft = Param(initialize=3000)                        # Tarifa de flete por km
model.c = Param(initialize=60.42)                         # Consumo combustible por km
model.r = Param(model.D, initialize={d: destinos[d]["d"] for d in destinos})                  # Demanda del municipio


Variables

In [177]:
model.x = Var(model.A, model.V, domain=Binary,initialize=0)               # Ruta tomada
model.fuel = Var(model.D, model.V, domain=NonNegativeReals,initialize=0)  # Nivel de combustible en el destino
model.u = Var(model.D | model.P, model.V, within=NonNegativeIntegers, bounds=(0, len(model.D | model.P) - 1)) # Variable auxiliar para evitar subtours
model.q = Var(model.V,within=NonNegativeIntegers)
model.fin = Var(model.V, model.P, within=Binary) # Nueva variable auxiliar




Funcion objetivo

In [178]:
def funcion_objetivo(model):
    travel_cost = sum(model.x[i,j,v] * (model.ft + model.m + model.c) * model.d[(i,j)] for (i,j) in model.A for v in model.V)
    return travel_cost

model.obj = Objective(rule=funcion_objetivo, sense=minimize)


In [179]:
nodos = list(depositos.keys()) + list(destinos.keys())

Restricciones

In [180]:
# 1. Conservación de flujo
def flow_conservation_rule(model, j, v):
    if j in model.P:
        return Constraint.Skip  # No se aplica en depósitos (son origen o destino)

    return (
        sum(model.x[i, j, v] for i in model.P | model.D if (i, j) in model.A)
        ==
        sum(model.x[j, k, v] for k in model.P | model.D if (j, k) in model.A)
    )
model.flow_conservation = Constraint(model.D, model.V, rule=flow_conservation_rule)

# Restricción 4: autonomía de vehículos
def autonomia_vehiculos_rule(model, v):
  return sum(model.d[i, j] * model.x[i, j, v] for (i, j) in model.A) <= model.a[v]
model.autonomia_vehiculos = Constraint(model.V, rule=autonomia_vehiculos_rule)

# 5. Capacidad del vehículo
def cap_limit_rule(model, v):
    return model.q[v] <= model.cap[v]
model.cap_limit = Constraint(model.V, rule=cap_limit_rule)

# Restriccion de todos los vecinos visitados
def visitar_clientes_rule(model, c):
  return sum(model.x[i, c, v] for v in model.V for i in nodos if (i, c) in model.A) == 1
model.visitar_clientes = Constraint(model.D, rule=visitar_clientes_rule)

# 8. Restricción de demanda
def demand_limit_rule(model, v):
    return sum(model.x[i, j, v] * model.r[j] for (i, j) in model.A if j not in model.P) <= model.cap[v]

model.demand_limit = Constraint(model.V, rule=demand_limit_rule)


# Eliminacion de subtours

def subtours_elimination_rule(model, i, j, v):
  if i != j and i in model.D and j in model.D:
    return model.u[i, v] - model.u[j, v] + len(nodos) * model.x[i, j, v] <= len(nodos) - 1
  else:
    return Constraint.Skip
model.subtours_elimination = Constraint(model.D, model.D, model.V, rule=subtours_elimination_rule)


# Unica salida de depositos

def salida_unica_rule(model, v):
  return sum(model.x[p, j, v] for p in model.P for j in model.D if (p, j) in model.A) == 1
model.salida_unica = Constraint(model.V, rule=salida_unica_rule)

# Retorno a un deposito
def retorno_centro_rule(model, v):
  return sum(model.x[i, p, v] for p in model.P for i in model.D if (i, p) in model.A) == 1
#model.retorno_centro = Constraint(model.V, rule=retorno_centro_rule)



def unico_retorno_rule(model, v):
    return sum(model.fin[v, p] for p in model.P) == 1

def fin_implica_llegada(model, v, p):
    return sum(model.x[i, p, v] for i in model.D if (i, p) in model.A) >= model.fin[v, p]

def orden_maximo_en_deposito(model, v, p):
    return model.u[p, v] >= len(model.D) * model.fin[v, p]


model.unico_retorno = Constraint(model.V, rule=unico_retorno_rule)
model.fin_implica_llegada = Constraint(model.V, model.P, rule=fin_implica_llegada)
model.orden_maximo_en_deposito = Constraint(model.V, model.P, rule=orden_maximo_en_deposito)




In [181]:
results = solver.solve(model, timelimit=600)
model.x.display()

x : Size=4800, Index=A*V
    Key                                        : Lower : Value : Upper : Fixed : Stale : Domain
     ('Deposito 1', 'Destino 1', 'Vehiculo 1') :     0 :   0.0 :     1 : False : False : Binary
     ('Deposito 1', 'Destino 1', 'Vehiculo 2') :     0 :   0.0 :     1 : False : False : Binary
     ('Deposito 1', 'Destino 1', 'Vehiculo 3') :     0 :   0.0 :     1 : False : False : Binary
     ('Deposito 1', 'Destino 1', 'Vehiculo 4') :     0 :   0.0 :     1 : False : False : Binary
     ('Deposito 1', 'Destino 1', 'Vehiculo 5') :     0 :   0.0 :     1 : False : False : Binary
     ('Deposito 1', 'Destino 1', 'Vehiculo 6') :     0 :   0.0 :     1 : False : False : Binary
     ('Deposito 1', 'Destino 1', 'Vehiculo 7') :     0 :   0.0 :     1 : False : False : Binary
     ('Deposito 1', 'Destino 1', 'Vehiculo 8') :     0 :   0.0 :     1 : False : False : Binary
    ('Deposito 1', 'Destino 10', 'Vehiculo 1') :     0 :   0.0 :     1 : False : False : Binary
    ('Deposito 

In [182]:
# Este codigo lo saque del proyecto que nos pasaron, pero mejor representar la info por otro metodo
print("Asignaciones ordenadas por vehículo:\n")
for v in model.V:
  salida = [(i,j) for (i,j) in model.A
              if i in model.D and model.x[i,j,v].value == 1]
  if not salida:
      print(f"Vehículo {v}: no sale de ningún centro")
      continue

  start_arc = salida[0]
  ruta = [ start_arc[0] ]
  actual = start_arc[1]
  ruta.append(actual)

  while actual not in model.D:
      siguiente = next((j for (i,j) in model.A
        if i==actual and model.x[i,j,v].value == 1),
                                None)
      if siguiente is None:
                    break
      ruta.append(siguiente)
      actual = siguiente

  print(f"Vehículo {v}:")
  for u, w in zip(ruta, ruta[1:]):
                print(f"  - {u} -> {w}")
  print()

Asignaciones ordenadas por vehículo:

Vehículo Vehiculo 1:
  - Destino 20 -> Deposito 1
  - Deposito 1 -> Destino 20

Vehículo Vehiculo 2:
  - Destino 9 -> Destino 17

Vehículo Vehiculo 3:
  - Destino 14 -> Deposito 1
  - Deposito 1 -> Destino 14

Vehículo Vehiculo 4:
  - Destino 12 -> Deposito 1
  - Deposito 1 -> Destino 12

Vehículo Vehiculo 5:
  - Destino 6 -> Deposito 1
  - Deposito 1 -> Destino 6

Vehículo Vehiculo 6:
  - Destino 23 -> Deposito 1
  - Deposito 1 -> Destino 23

Vehículo Vehiculo 7:
  - Destino 3 -> Deposito 1
  - Deposito 1 -> Destino 21

Vehículo Vehiculo 8:
  - Destino 10 -> Destino 4

